### Naïve Bayes Mixture Model

&nbsp;

Gaussian Mixture Model is great but text doesn't follow Gaussian distribution. We need another algorithm for text clustering. If you recall generative learning algorithms, we have Naïve Bayes apart from Gaussian Discriminant Analysis. It should not come as a surprise that we have Naïve Bayes Mixture Model as well. Mathematically, it is somewhat similar to Naïve Bayes. In Naïve Bayes, we can predict a new data point based upon Bayes' rule where p(y=label|x)=p(x|y=label)\*p(y=label)/p(x). The concept holds in NBMM. To make our life easier, the following script only takes the case of multivariate, to be more precise, a binary cluster problem. I will leave you guys to explore multinomial NBMM.

Like GMM, p(y) remains a latent variable we do not observe. In order to estimate p(y), we need to take an initial guess and gradually converge to the optimal number throughout iterations. Undoubtedly, we use Expectation-Maximization algorithm as well.

The material from Stanford university doesn't really cover the details of NBMM. It has been briefly introduced in Andrew Ng's lecture video. If you fully comprehend the concept of Naïve Bayes and Gaussian Mixture Model, the equations can be easily derived. The only issue is the lack of literature on how to select the optimal number of clusters in NBMM. Since it's distribution-based, I assume you can use AIC and BIC as GMM. I will leave you guys to explore.

Reference to Gaussian Mixture Model

https://github.com/je-suis-tm/machine-learning/blob/master/gaussian%20mixture%20model.ipynb

Reference to Naïve Bayes

https://github.com/je-suis-tm/machine-learning/blob/master/naive%20bayes.ipynb

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn.datasets
import sklearn.decomposition
import pandas as pd
import os
import nltk.tokenize
import nltk.corpus
import nltk.stem

In [2]:
os.chdir('K:/ecole/github')

In [3]:
#convert text into a list of words
#we can use stemming and lemmatization to improve efficiency
#for instance, we have words walked,walking,walks
#with nltk package, we can revert all of them to walk
def text2list(text,stopword,lower=True,
              lemma=False,stemma=False):

    text_clean=text if lower==False else text.lower()
    
    #tokenize and remove stop words
    token=[i for i in nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text_clean) if i not in stopword]
    
    #lemmatization
    if lemma:
        text_processed=[nltk.stem.wordnet.WordNetLemmatizer().lemmatize(i) for i in token]
    else:
        text_processed=token
        
    #stemming
    if stemma:
        output=[nltk.stem.PorterStemmer().stem(i) for i in text_processed]
    else:
        output=text_processed
    
    #remove numbers as they are stopword as well
    for i in [ii for ii in output]:
        try:
            float(i)
            output.remove(i)
        except:
            pass
    
    return [i for i in output if i not in stopword]

##### E Step

&nbsp;

\begin{equation*}
    \begin{split}
        w^{(i)}:=\frac {P(X^{(i)}|Z_1) \times P(Z_1)} {\sum_{k=0}^{1} {P(X^{(i)}|Z_k) \times P(Z_k)}}=\frac{\prod_{j=0}^{n} {P(x_j^{(i)}|Z_1)} \times P(Z_1)}
        {\prod_{j=0}^{n}{P(x_j^{(i)}|Z_1)} \times P(Z_1) + \prod_{j=0}^{n} {P(x_j^{(i)}|Z_0)} \times P(Z_0)}
    \end{split}
\end{equation*}

In [4]:
#e step
#compute the posterior probability with given params
def e_step(matrix,params):

    posterior=[[],[]]

    #unpacking
    prob_x_given_z,prob_z_0,prob_z_1=params

    #transpose the matrix
    transpose=matrix.T
    transpose.drop('PRIOR',inplace=True)

    #iterate through all datasets
    for i in transpose:
        phi_x_given_z_0=1.0
        phi_x_given_z_1=1.0

        #chain rules of independent events
        for j in transpose[transpose[i]==1].index:
            phi_x_given_z_0*=prob_x_given_z[j][0]
            phi_x_given_z_1*=prob_x_given_z[j][1]

        #when the probability gets too small
        #python gives zero
        #we d rather have ham email rather than spam email
        #fear of missing out
        if phi_x_given_z_0==0 or phi_x_given_z_1==0:
            phi_z_0=1
            phi_z_1=0
        else:
            phi_z_0=phi_x_given_z_0*prob_z_0/(phi_x_given_z_0*prob_z_0+phi_x_given_z_1*prob_z_1)
            phi_z_1=phi_x_given_z_1*prob_z_1/(phi_x_given_z_0*prob_z_0+phi_x_given_z_1*prob_z_1)

        posterior[0].append(phi_z_0)
        posterior[1].append(phi_z_1)

    return posterior

##### M Step

&nbsp;

\begin{equation*}
    \begin{split}
    P(x_j|Z_1)=\frac{\sum_{i=0}^{m} {w^{(i)} \times 1\{x_j^{(i)}=1\}}} {\sum_{i=0}^{m}{w^{(i)}}}    
    \end{split}
    \\
    \begin{split}
    P(x_j|Z_0)=\frac{\sum_{i=0}^{m} {(1-w^{(i)}) \times 1\{x_j^{(i)}=1\}}} {\sum_{i=0}^{m} {(1-w^{(i)})}}
    \end{split}
    \\
    \begin{split}
    P(Z_1)=\frac{\sum_{i=0}^{m}{w^{(i)}}}{m}
    \end{split}
    \\
    \begin{split}
    P(Z_0)=\frac{\sum_{i=0}^{m} {(1-w^{(i)})}}{m}
    \end{split}
\end{equation*}

In [5]:
#m step
#use posterior probability to update params
def m_step(matrix):

    #compute P(x_j|Z)
    prob_x_given_z=pd.DataFrame(columns=matrix.columns)
    for j in matrix.columns:

        #compute the frequency of word j coinciding with the given cluster
        freq_j_given_z_1=len(matrix[matrix[j]==1][matrix['PRIOR']==1])
        freq_j_given_z_0=len(matrix[matrix[j]==1][matrix['PRIOR']==0])

        #laplace smoothing
        if freq_j_given_z_1==0:
            prob_x_given_z_1=1/(len(matrix)+2)
        else:
            prob_x_given_z_1=freq_j_given_z_1/len(matrix)
        if freq_j_given_z_0==0:
            prob_x_given_z_0=1/(len(matrix)+2)
        else:
            prob_x_given_z_0=freq_j_given_z_0/len(matrix)

        #create dataframe
        prob_x_given_z[j]=[prob_x_given_z_0,prob_x_given_z_1]

    #compute P(Z)
    prob_z_1=matrix['PRIOR'].sum()/len(matrix)
    prob_z_0=1-prob_z_1

    return (prob_x_given_z,prob_z_0,prob_z_1)

##### Lower Bound

&nbsp;

\begin{equation*}
    \begin{split}
    l(\theta) \ge \sum_{i=0}^{m} { \sum_{k=0}^{1} {w^{(i)} \times \log \frac {P(X^{(i)}|Z_k) \times P(Z_k)} {w^{(i)}} } } = \sum_{i=0}^{m} { \sum_{k=0}^{1} {w^{(i)} \times (\log P(X^{(i)}|Z_k) + \log P(Z_k) - \log {w^{(i)}}) } }
    \end{split}
\end{equation*}

In [6]:
#using jensens inequality to compute the lower bound
#which is the function of the expectation
def get_lower_bound(matrix,posterior,params):

    phi_z_0=[];phi_z_1=[]
    
    #catch logarithm zero issue
    np.seterr(divide='raise')

    #unpacking
    prob_x_given_z,prob_z_0,prob_z_1=params

    #transpose the matrix
    transpose=matrix.T
    transpose.drop('PRIOR',inplace=True)

    #iterate through all datasets
    for i in transpose:
        phi_x_given_z_0=1.0
        phi_x_given_z_1=1.0

        #chain rules of independent events
        for j in transpose[transpose[i]==1].index:
            phi_x_given_z_0*=prob_x_given_z[j][0]
            phi_x_given_z_1*=prob_x_given_z[j][1]

        #when the probability gets too small
        #python gives zero
        #logarithm does not take zero
        #to avoid that, use 1 instead
        #so no effect on lower bound
        if phi_x_given_z_0==0:
            phi_x_given_z_0=1
        if phi_x_given_z_1==0:
            phi_x_given_z_1=1

        phi_z_0.append(phi_x_given_z_0)
        phi_z_1.append(phi_x_given_z_1)

    #logarithm does not take zero
    #to avoid that, use 1 instead
    #so no effect on lower bound
    for i in range(2):
        for ind,val in enumerate(posterior[i]):
            if val==0:
                posterior[i][ind]=1

    #sum up
    summation_0=posterior[0]*(np.log(phi_z_0)+np.log(prob_z_0)-np.log(posterior[0]))
    summation_1=posterior[1]*(np.log(phi_z_1)+np.log(prob_z_1)-np.log(posterior[1]))

    return sum(summation_0+summation_1)

In [7]:
#using mle for training
def training(matrix,tolerance=0.001,num_of_itr=50,diagnosis=False):
    
    #initialize
    lower_bound_old=None
    lower_bound=None
    counter=0        
    
    #use m step to get params
    params=m_step(matrix)

    #cap the maximum number of iterations
    while counter<num_of_itr:        
                                            
        #e step
        posterior=e_step(matrix,params)
            
        #update prior
        matrix['PRIOR']=np.where(np.array(posterior[0])>np.array(posterior[1]),0,1)

        #m step
        params=m_step(matrix)
        
        #use lower bound to determine if converged
        lower_bound_old=lower_bound
        lower_bound=get_lower_bound(matrix,posterior,params)
        if lower_bound_old and np.abs(lower_bound/lower_bound_old-1)<tolerance:
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')
            return params
                        
        counter+=1
           
    if diagnosis:
        print(f'{counter} iterations to reach convergence\n')
        
    return params

In [8]:
#the main function
def nbmm(df,tolerance=0.001,num_of_itr=50,diagnosis=False):
    
    data=df.copy()
    data['y']=data['spam']
    
    #tokenization
    tokens=[]
    for i in data['text'].tolist():
        tokens.append(text2list(i,stopword,
                                lower=True,lemma=True))            
    data['word']=tokens

    #get vocabulary
    vocabulary=set([j for i in data['word'] for j in i if i not in stopword])

    #using dataframe to create vocabulary matrix
    #it is slow but more convenient
    matrix=pd.DataFrame(columns=vocabulary)
    for i in vocabulary:
        temp=[]
        for j in range(len(data)):
            if i in data['word'][j]:
                temp.append(1)  
            else:
                temp.append(0)
        matrix[i]=temp

    #initial parameters
    #use random prior probability
    prior=[np.random.randint(low=0,high=2) for i in range(len(df))]
    matrix['PRIOR']=prior
    
    #obtain parameters of nbmm
    params=training(matrix,tolerance,num_of_itr,diagnosis)
    
    #compute the prediction probability
    posterior=e_step(matrix, params)
    
    #make forecast based upon probability
    data['label']=np.where(np.array(posterior[0])>np.array(posterior[1]),0,1)
    
    #compute accuracy
    erreur=0
    checked=[]
    for i in data['y'].unique():
        erreur+=get_accuracy(data,i,checked)
        checked.append(i)
    accuracy=1-erreur/len(df)
    if diagnosis:
        print('\naccuracy: %s'%(accuracy))
    
    return params,posterior

In [9]:
#for unsupervised learning, clf.score doesnt return the accuracy
#there is no cross validation, no known labels
#the only way to detect the accuracy is vote of the majority
#for each label given
#we check which iris type is the majority
#we consider the majority as the correct classification
#all we need to do is to count the minority
def get_accuracy(data,class_,checked):
    
    df=data.copy()
    
    #use dictionary to keep track of everything
    d={}
    
    #counting
    for i in df['label'][df['y']==class_].unique():
        if i not in checked and i!=-1:
            d[i]=df['label'][df['y']==class_].tolist().count(i)

    #comparison
    temp=-1
    lbl=None
    for i in d:
        if d[i]>temp:
            lbl=i
            temp=d[i]

    return len(df['label'][df['y']==class_][df['label']!=lbl])

In [10]:
stopword=nltk.corpus.stopwords.words('english')+['u',
 'beyond',
 'within',
 'around',
 'would',
 'b',
 'c',
 'e',
 'f',
 'g',
 'h',
 'j',
 'k',
 'l',
 'n',
 'p',
 'q',
 'r',
 'u',
 'v',
 'w',
 'x',
 'z',
 'first']

In [11]:
#the raw data really comes from my email
df=pd.read_csv('spam.csv')

In [12]:
params,posterior=nbmm(df,diagnosis=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


2 iterations to reach convergence


accuracy: 0.5833333333333333
